# Анализ метрик на датасете Wikidata Big (Temporal Knowledge Graph)

Этот блокнот демонстрирует процесс адаптации существующей системы к работе с крупномасштабным темпоральным графом знаний **Wikidata Big**. 

**Основные шаги:**
1. Загрузка данных Wikidata (сущности, отношения, временные метки).
2. Отображение (Mapping) числовых ID в строковые представления Wikidata (Q-IDs, P-IDs).
3. Инициализация векторной модели и индексация темпоральных триплетов.
4. Расчет метрик качества (Hits@K, MRR) с учетом временной координаты.

In [1]:
import sys
import os
import pickle
import numpy as np
import random
import shutil
from tqdm.auto import tqdm
import torch

# Добавление src в путь для импорта модулей проекта
sys.path.append('../../')

# Импорт необходимых компонентов системы
from src.kg_model.embeddings_model import EmbeddingsModel, EmbeddingsModelConfig, EmbedderModelConfig
from src.db_drivers.vector_driver import VectorDriverConfig, VectorDBConnectionConfig, VectorDBInstance
from src.utils.data_structs import TripletCreator, NodeCreator, NodeType, RelationCreator, RelationType
from src.utils import Logger

ModuleNotFoundError: No module named 'pysqlite3'

## 1. Загрузка и подготовка данных Wikidata

Данные WikidataBig хранятся в формате pickle и содержат:
- `ent_id`: словарь соответствия Q-идентификаторов (строк) числовым ID.
- `rel_id`: словарь соответствия P-идентификаторов (отношений) числовым ID.
- `ts_id`: словарь временных меток.
- `test.pickle` / `valid.pickle`: массивы триплетов с временными интервалами.

In [ ]:
DATA_PATH = '../../wikidata_big/kg/tkbc_processed_data/wikidata_big/'

def load_wikidata_mapping(file_name):
    with open(os.path.join(DATA_PATH, file_name), 'rb') as f:
        return pickle.load(f)

print("Загрузка словарей соответствия...")
ent_to_id = load_wikidata_mapping('ent_id')
rel_to_id = load_wikidata_mapping('rel_id')
ts_to_id = load_wikidata_mapping('ts_id')

# Создаем обратные словари для восстановления строк по ID
id_to_ent = {v: k for k, v in ent_to_id.items()}
id_to_rel = {v: k for k, v in rel_to_id.items()}
id_to_ts = {v: str(k) for k, v in ts_to_id.items()}

print(f"Загружено сущностей: {len(ent_to_id)}")
print(f"Загружено отношений: {len(rel_to_id)}")
print(f"Загружено временных меток: {len(ts_to_id)}")

In [ ]:
print("Загрузка тестовых данных...")
with open(os.path.join(DATA_PATH, 'test.pickle'), 'rb') as f:
    test_data = pickle.load(f) # Формат: (s, r, o, start_t, end_t)

print(f"Количество тестовых квадруплетов: {len(test_data)}")

## 2. Конвертация в формат системы (Triplet)

Мы преобразуем числовые данные Wikidata в объекты `Triplet`, используя нашу обновленную структуру с поддержкой времени.

In [ ]:
def convert_to_triplets(data_subset, sample_limit=500):
    # Берем подмножество для ускорения демонстрации
    subset = data_subset[:sample_limit]
    converted = []
    
    for row in tqdm(subset, desc="Конвертация"):
        # Извлекаем названия из ID
        s_name = id_to_ent[row[0]]
        r_name = id_to_rel[row[1]]
        o_name = id_to_ent[row[2]]
        t_name = id_to_ts[row[3]] # Используем стартовое время как основной маркер
        
        # Создаем узлы и отношение
        s_node = NodeCreator.create(NodeType.object, s_name)
        r_rel = RelationCreator.create(RelationType.simple, r_name)
        o_node = NodeCreator.create(NodeType.object, o_name)
        t_node = NodeCreator.create(NodeType.time, t_name)
        
        # Создаем темпоральный триплет (S, P, O, T)
        triplet = TripletCreator.create(s_node, r_rel, o_node, time=t_node)
        converted.append(triplet)
        
    return converted

test_triplets = convert_to_triplets(test_data)
print(f"Пример: {test_triplets[0].stringified}")

## 3. Инициализация модели и расчет метрик

Мы инициализируем `EmbeddingsModel` и проводим поиск по векторизованному представлению запроса `(s, p, t)` для нахождения верного `o`.

In [ ]:
# Конфигурация векторного хранилища (тестовая)
NODES_DB_PATH = '../../data/graph_structures/vectorized_nodes/wikidata_test'
TRIPLETS_DB_PATH = '../../data/graph_structures/vectorized_triplets/wikidata_test'
EMBEDDER_PATH = '../../models/intfloat/multilingual-e5-small'
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# Очистка предыдущих тестов
for path in [NODES_DB_PATH, TRIPLETS_DB_PATH]:
    if os.path.exists(path): shutil.rmtree(path)

config = EmbeddingsModelConfig(
    nodesdb_driver_config=VectorDriverConfig(db_config=VectorDBConnectionConfig(path=NODES_DB_PATH, need_to_clear=True)),
    tripletsdb_driver_config=VectorDriverConfig(db_config=VectorDBConnectionConfig(path=TRIPLETS_DB_PATH, need_to_clear=True)),
    embedder_config=EmbedderModelConfig(model_name_or_path=EMBEDDER_PATH, device=DEVICE)
)

model = EmbeddingsModel(config)
model.embedder.init_model()

# Индексация данных
print("Индексация тестовых триплетов в векторную БД...")
model.create_triplets(test_triplets)

## 4. Расчет MRR и Hits@K

**Метрики:**
- **Hits@K**: Доля случаев, когда правильный ответ находится в топ-K результатах.
- **MRR (Mean Reciprocal Rank)**: Среднее значение величины, обратной рангу правильного ответа.

In [ ]:
def evaluate_wikidata_link_prediction(model, triplets, k_values=[1, 5, 10]):
    hits = {k: 0 for k in k_values}
    mrr = 0
    count = 0
    
    for triplet in tqdm(triplets, desc="Оценка"):
        # Формируем запрос: "Time: {t} | Subject Relation"
        t_str = triplet.time.name
        s_str = triplet.start_node.name
        r_str = triplet.relation.name
        query_text = f"{t_str}: {s_str} {r_str}"
        
        # Получаем эмбеддинг запроса
        q_emb = model.embedder.encode_passages([query_text])[0]
        query_inst = VectorDBInstance(id='q', document=query_text, embedding=q_emb)
        
        # Поиск ближайших узлов (кандидатов на роль Object)
        results = model.vectordbs['nodes'].retrieve([query_inst], n_results=100, includes=['ids'])
        candidates = results[0] # Список (distance, instance)
        
        # Ищем ранг правильного объекта
        gold_id = triplet.end_node.id
        rank = None
        for i, (dist, inst) in enumerate(candidates):
            if inst.id == gold_id:
                rank = i + 1
                break
        
        if rank is not None:
            mrr += 1.0 / rank
            for k in k_values:
                if rank <= k: hits[k] += 1
        
        count += 1
    
    # Усреднение результатов
    mrr /= count
    final_hits = {k: v / count for k, v in hits.items()}
    
    return final_hits, mrr

hits, mrr = evaluate_wikidata_link_prediction(model, test_triplets)

print("\n--- РЕЗУЛЬТАТЫ WIKIDATA BIG ---")
print(f"MRR: {mrr:.4f}")
for k, val in hits.items():
    print(f"Hits@{k}: {val:.4f}")